<a href="https://colab.research.google.com/github/sanath8107/strategies/blob/main/simple_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A simple backtesting method that takes in the strategey, data as the input and the gives the profit generated from the strategy

More features need to be added such as the confusion matrix, investment required

Author: Chandan

# Strategy

In [ ]:
#######################  STRATEGY #######################

def macd(data):
    data = clean_data(data, 'close')
    short_ema = data['close'].ewm(span=12, adjust=False).mean()
    # Calculate the long term exponential moving average(EMA)
    long_ema = data['close'].ewm(span=26, adjust=False).mean()
    # Calculate the MACD line
    macd_value = short_ema - long_ema
    # Calculate the signal line
    signal = macd_value.ewm(span=9, adjust=False).mean()

    df = pd.DataFrame(list(zip(macd_value, signal)),
                      columns=['MACD', 'Signal Line'])

    buy = []
    sell = []
    flag = -1

    for i in range(0, len(df)):
        if df['MACD'][i] > df['Signal Line'][i]:
            sell.append(np.nan)
            if flag != 1:
                buy.append(data['close'][i])
                flag = 1
            else:
                buy.append(np.nan)
        elif df['MACD'][i] < df['Signal Line'][i]:
            buy.append(np.nan)
            if flag != 0:
                sell.append(data['close'][i])
                flag = 0
            else:
                sell.append(np.nan)
        else:
            buy.append(np.nan)
            sell.append(np.nan)
    buy_sell_points = {'buy': buy, 'sell': sell}
    return pd.DataFrame.from_dict(buy_sell_points)

#########################################################

# Get Data

In [ ]:
import sys

import requests
import csv
import datetime
import pandas as pd
import numpy as np

cookie_url = 'https://www.nseindia.com/get-quotes/derivatives?symbol=BANKNIFTY'
equity_base_url = 'https://www.nseindia.com/api/historical/cm/equity'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/87.0.4280.88 Safari/537.36'}


def get_urls(scrip, from_date, to_date, instrument_type):
    url_list = []

    # temp_from_date = from_date
    from_date = datetime.datetime.strptime(from_date, '%d-%m-%Y')
    to_date = datetime.datetime.strptime(to_date, '%d-%m-%Y')

    day_diff = (to_date - from_date).days
    if day_diff < 0:
        print("From date should be earlier")
        sys.exit(0)

    temp_date = from_date + datetime.timedelta(days=365 * 2)
    while temp_date <= to_date:
        equity_data_url = equity_base_url
        equity_data_url += '?' + 'symbol=' + scrip.upper()
        equity_data_url += '&' + 'series=' + '[%22' + instrument_type + '%22]'
        equity_data_url += '&' + 'from=' + from_date.strftime("%d-%m-%Y")
        equity_data_url += '&' + 'to=' + temp_date.strftime("%d-%m-%Y")
        equity_data_url += '&' + 'csv=true'
        url_list.append(equity_data_url)
        # print(equity_data_url)
        from_date = temp_date
        temp_date = from_date + datetime.timedelta(days=365 * 2)

    equity_data_url = equity_base_url
    equity_data_url += '?' + 'symbol=' + scrip.upper()
    equity_data_url += '&' + 'series=' + '[%22' + instrument_type + '%22]'
    equity_data_url += '&' + 'from=' + from_date.strftime("%d-%m-%Y")
    equity_data_url += '&' + 'to=' + to_date.strftime("%d-%m-%Y")
    equity_data_url += '&' + 'csv=true'
    url_list.append(equity_data_url)
    # print(equity_data_url)

    return url_list


def get_data(scrip, from_date, to_date, instrument_type):
    url_list = get_urls(scrip, from_date, to_date, instrument_type)
    full_data = []

    with requests.session() as s:
        # load cookies:
        s.get(cookie_url, headers=headers)

        for url in url_list:
            _data = s.get(url, headers=headers)  # .json()
            decoded_content = _data.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            my_list = list(cr)
            my_list.reverse()
            title_row = my_list.pop()
            for row in my_list:
                full_data.append(row)

    # convert to dataframe
    title_row[0] = "Date"
    title_row = [title.strip().lower() for title in title_row]
    full_data = pd.DataFrame(
        data=full_data,
        columns=title_row
    )
    full_data.set_index(pd.DatetimeIndex(full_data['date'].values))
    print("Data collection complete, days =", len(full_data))
    return full_data

def clean_data(data, which):
    if which == 'close':
        data['close'] = data['close'].apply(lambda x: x.replace(',',''))
        return data


# Backtest and Result

In [ ]:
class Result():
    def __init__(self, data, buy_sell):
        self.profit = 0
        self.profit_percentage = 0
        self.true_positive = 0
        self.false_positive = 0
        self.true_negative = 0
        self.false_negative = 0
        self.data = data
        self.buy_sell = buy_sell
        self.run()

    def run(self):
        first_buy = self.buy_sell['buy'].first_valid_index()
        last_sell = self.buy_sell['sell'].last_valid_index()
        for i in range(first_buy, last_sell+1):
            if self.buy_sell['buy'][i] is not np.nan and self.buy_sell['sell'][i] is np.nan:
                self.profit -= float(self.buy_sell['buy'][i])
            elif self.buy_sell['buy'][i] is np.nan and self.buy_sell['sell'][i] is not np.nan:
                self.profit += float(self.buy_sell['sell'][i])


def backtest(data, strategy, data_needed):
    if not set(data_needed).issubset(set(data.columns)):
        raise ValueError("data needed could not be collected")
    data_needed = data[data_needed]
    buy_sell_points = strategy(data_needed)
    result = Result(data, buy_sell_points)
    return result

# Driver Code

In [ ]:
data = get_data('WIPRO', '01-01-2017', '01-01-2018', 'EQ')

try:
    result = backtest(data, macd, ['date', 'close'])
    print("Profit Generated: ", result.profit)
except Exception as ex:
    print(ex)
